In [ ]:
from typing import Annotated

from langchain_core.documents import Document
from langchain_core.messages import SystemMessage,HumanMessage
from langchain_core.tools import tool
from typing_extensions import List, TypedDict

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

from IPython.display import Image, display

from tools import llm, custom_llm_with_tools

from VectorDB import VectorDB

In [ ]:
PERSIST_DIR = "./chroma_langchain_db"
COLLECTION_NAME = "movies_collection"

# Initialize vector database
vector_db = VectorDB(model_name="BAAI/bge-base-en-v1.5", batch_size=32)
init_result = vector_db.initialize_vector_store(PERSIST_DIR, COLLECTION_NAME)

vector_store = vector_db.vector_store

In [ ]:
generator_prompt = (
    "You are a movie recommendation / finding assistant, your job is to help users find movies based on their preferences."
    "You will start by reading the user's query and then searching for relevant movies in the database."
    "Then you will use the available context to provide a personalized recommendation of a movie title:" \
    "- Identify the main 3 genres of the movie from the context"
    "- Identify the the release year, director, and main actors of the movie from the context"
    "- Identify the main themes and plotline of the movie from the context"
    "When generating the final response, make sure to include all relevant information in the following order:"
    "1 - Title:"
    "2 - Genres:"
    "3 - Release Year, Director, and Main Actors:"
    "4 - Themes and Plotline:"
    "Query:\n{question}"
    "Context:\n{context}\n"
)

GENERATOR_PROMPT = PromptTemplate.from_template(generator_prompt)

class State(MessagesState):
    context: List[Document]


class MovieRecommendationWithSources(TypedDict):
    """A movie recommendation with detailed information and sources."""
    
    movie_title: str
    genres: Annotated[
        List[str], 
        "Main 3 genres of the recommended movie"
    ]
    release_year: Annotated[
        int,
        "Year the movie was released"
    ]
    director: Annotated[
        str,
        "Director of the movie"
    ]
    main_actors: Annotated[
        List[str],
        "List of main actors in the movie"
    ]
    themes_and_plot: Annotated[
        str,
        "Brief description of main themes and plotline"
    ]
    recommendation_reason: Annotated[
        str,
        "Detailed explanation of why this movie matches the user's preferences"
    ]
    sources: Annotated[
        List[str],
        "List of source documents/databases used to gather this movie information"
    ]

def query_or_respond(state: State):
    """Generate tool call for movie retrieval or respond directly."""
    
    # Add system message to encourage tool usage for movie queries
    system_message = SystemMessage(content=(
        "You are a movie recommendation assistant. When users ask about movies, "
        "you should use the retrieve tool to search the movie database "
        "before providing recommendations. Only recommend movies found in the database."
    ))
    
    # Combine system message with conversation history
    messages_with_system = [system_message] + state["messages"]
    
    # Bind tools to the LLM
    llm_with_tools = llm.bind_tools([retrieve])
    response = llm_with_tools.invoke(messages_with_system)
    
    return {"messages": [response]}

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    # Increase k to get more movies when user asks for multiple recommendations
    retrieved_docs = vector_store.similarity_search(query, k=6)  # Increased from 2 to 6
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

tools = ToolNode([retrieve])

def generate(state: MessagesState):
    """Generate structured movie recommendation with sources."""
    
    # Get the most recent tool messages (movie retrieval results)
    recent_tool_messages = []
    for message in reversed(state["messages"]):
        if message.type == "tool":
            recent_tool_messages.append(message)
        else:
            break
    tool_messages = recent_tool_messages[::-1]  # Reverse to get correct order
    
    # Format movie information for the context
    movies_content = "\n\n".join(msg.content for msg in tool_messages)
    
    # Get the most recent user question
    user_question = ""
    for message in reversed(state["messages"]):
        if message.type == "human":
            user_question = message.content
            break
    
    # Enhanced prompt for better recommendations with explicit field requirements
    enhanced_prompt = f"""You are a movie recommendation assistant. Based on the user's query and the provided movie database context, provide a detailed movie recommendation.

User Query: {user_question}

Available Movies from Database:
{movies_content}

IMPORTANT: You must fill out ALL fields in your response:
- movie_title: The exact title of the movie
- genres: List the main 3 genres (e.g., ["Science Fiction", "Thriller", "Drama"])
- release_year: The year as a number
- director: Full name of the director
- main_actors: List of main actor names (e.g., ["Harrison Ford", "Rutger Hauer", "Sean Young"])
- themes_and_plot: Detailed description of the plot and main themes
- recommendation_reason: Explain why this movie matches the user's request
- sources: List of sources used (e.g., ["Movie Database", "IMDb"])

Only recommend ONE movie that is mentioned in the database context above. Extract all information from the provided context."""

    try:
        # Use structured output for movie recommendations
        structured_llm = llm.with_structured_output(MovieRecommendationWithSources)
        response = structured_llm.invoke(enhanced_prompt)
        
        # Debug: print what we got
        print("DEBUG - Structured response:", response)
        
        # Safely extract fields with defaults
        movie_title = response.get('movie_title', 'Unknown Movie')
        release_year = response.get('release_year', 'Unknown')
        director = response.get('director', 'Unknown Director')
        genres = response.get('genres', ['Unknown Genre'])
        main_actors = response.get('main_actors', ['Unknown Actor'])
        themes_and_plot = response.get('themes_and_plot', 'Plot information not available.')
        recommendation_reason = response.get('recommendation_reason', 'This movie was found in the database.')
        sources = response.get('sources', ['Movie Database'])
        
        # Ensure genres and main_actors are lists
        if isinstance(genres, str):
            genres = [genres]
        if isinstance(main_actors, str):
            main_actors = [main_actors]
        
        # Create verbose response
        verbose_response = f"""Based on your request, here's my recommendation from the database:

🎬 **{movie_title}** ({release_year})

**Director:** {director}
**Genres:** {', '.join(genres)}
**Cast:** {', '.join(main_actors)}

**Plot & Themes:**
{themes_and_plot}

**Why I recommend this movie:**
{recommendation_reason}

**Sources:** {', '.join(sources)}"""

    except Exception as e:
        print(f"Error in structured generation: {e}")
        # Fallback to simple response
        verbose_response = f"""Based on your request and the available database entries, I found relevant movies but encountered an issue with structured formatting. 

Here's what I found in the database:
{movies_content[:500]}...

Please try your request again or be more specific about what type of movie you're looking for."""
    
    # Extract context from tool message artifacts for state tracking
    context = []
    for tool_message in tool_messages:
        if hasattr(tool_message, 'artifact') and tool_message.artifact:
            context.extend(tool_message.artifact)
    
    return {
        "messages": [{"role": "assistant", "content": response}], 
        "context": context
    }

tools = ToolNode([retrieve])




#def chatbot(state: State):
#    return {"messages": [llm.invoke(state["messages"])]}

In [ ]:
#graph_builder = StateGraph(State)

#graph_builder.add_node("chatbot", chatbot)

#graph_builder.add_edge(START, "chatbot")
#graph_builder.add_edge("chatbot", END)

graph_builder = StateGraph(State)
    
# Add nodes
graph_builder.add_node("query_or_respond", query_or_respond)
graph_builder.add_node("tools", tools)
graph_builder.add_node("generate", generate)

# Set entry point
graph_builder.set_entry_point("query_or_respond")

# Add conditional edges
graph_builder.add_conditional_edges(
    "query_or_respond",
    tools_condition,
    {END: END, "tools": "tools"},
)

# Add regular edges
graph_builder.add_edge("tools", "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
result = graph.invoke({
    "messages": [{"role": "user", "content": "Recommend 3 good sci-fi movie from the 1980s."}]
})

# Access the final response
final_message = result["messages"][-1]
print(final_message.content)

# Access the retrieved context (movies that were found)
retrieved_movies = result["context"]
print(f"Found {len(retrieved_movies)} movies in the database")

In [ ]:
result.keys()

In [ ]:
result['messages']

In [ ]:
# Check if any tool calls were made
for msg in result["messages"]:
    if hasattr(msg, 'tool_calls') and msg.tool_calls:
        print(f"Tool calls made: {[tc['name'] for tc in msg.tool_calls]}")
    else:
        print(f"Message type {msg.type}: No tool calls")

print(f"Context available: {'context' in result}")

In [ ]:
result = retrieve("1980s sci-fi movies")
print("Tool result:", result)
print("Type:", type(result))

# If it returns a tuple (content, artifacts)
if isinstance(result, tuple):
    content, artifacts = result
    print("Content:", content)
    print("Artifacts count:", len(artifacts) if artifacts else 0)

In [ ]:
llm_with_tools = llm.bind_tools([retrieve])

# Create a more explicit message that should trigger tool usage
test_messages = [
    SystemMessage(content="You must use the retrieve tool to search for movies. Do not answer without using the tool."),
    HumanMessage(content="Find me 1980s sci-fi movies from the database")
]

response = llm_with_tools.invoke(test_messages)
print("Response type:", type(response))
print("Has tool calls:", hasattr(response, 'tool_calls') and bool(response.tool_calls))
if hasattr(response, 'tool_calls') and response.tool_calls:
    print("Tool calls:", response.tool_calls)
else:
    print("Response content:", response.content)

In [ ]:
print("LLM model:", type(llm))
print("LLM attributes:", dir(llm))

In [ ]:
try:
    llm_with_tools = llm.bind_tools([retrieve])
    print("Tool binding successful")
    
    # Check if the model has tool calling attributes
    print("Model attributes:", [attr for attr in dir(llm) if 'tool' in attr.lower()])
    
except Exception as e:
    print(f"Tool binding failed: {e}")

In [ ]:
# Let's see what bind_tools actually creates
llm_with_tools = llm.bind_tools([retrieve])
print("LLM with tools type:", type(llm_with_tools))
print("LLM with tools attributes:", [attr for attr in dir(llm_with_tools) if not attr.startswith('_')])

# Check if tools are properly bound
if hasattr(llm_with_tools, 'bound_tools'):
    print("Bound tools:", llm_with_tools.bound_tools)

In [ ]:
test_messages = [
    SystemMessage(content="You have access to a tool called 'retrieve'. Use it to search for movies."),
    HumanMessage(content="Call retrieve with query '1980s sci-fi'")
]

response = llm_with_tools.invoke(test_messages)
print("Explicit format - Tool calls:", getattr(response, 'tool_calls', None))

# Test 2: JSON-like instruction
test_messages2 = [
    HumanMessage(content="Please use the retrieve function to search for '1980s sci-fi movies'. Call: retrieve(query='1980s sci-fi movies')")
]

response2 = llm_with_tools.invoke(test_messages2)
print("JSON format - Tool calls:", getattr(response2, 'tool_calls', None))

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_name = "meta-llama/Llama-3.2-3B-Instruct"

print("Loading model and tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Check if your model has specific tool calling requirements
print("Model config:", llm.llm.pipeline.model.config)
print("Tokenizer chat template:", tokenizer.chat_template if hasattr(tokenizer, 'chat_template') else "No chat template")

In [ ]:
try:
    llm_forced = llm.bind_tools([retrieve], tool_choice="any")
    response = llm_forced.invoke([HumanMessage(content="Find 1980s sci-fi movies")])
    print("Forced tool choice - Tool calls:", getattr(response, 'tool_calls', None))
except Exception as e:
    print(f"Tool choice failed: {e}")

In [ ]:
test_message = """Given the following functions, please respond with a JSON for a function call with its proper arguments that best answers the given prompt.

Respond in the format {"name": function name, "parameters": dictionary of argument name and its value}. Do not use variables.

{
    "type": "function",
    "function": {
        "name": "retrieve",
        "description": "Retrieve movie information related to a user's preferences and query.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Search query for movies"
                }
            },
            "required": ["query"]
        }
    }
}

Find me sci-fi movies from the 1980s."""

response = llm.invoke([HumanMessage(content=test_message)])
print("Response:", response.content)

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

# Test basic tool calling
llm_with_tools = llm.bind_tools([retrieve])

test_messages = [
    SystemMessage(content="You are an assistant that must use tools when available. Use the retrieve_movies tool when asked about movies."),
    HumanMessage(content="Use the retrieve_movies tool to find sci-fi movies from the 1980s")
]

response = llm_with_tools.invoke(test_messages)
print("Response type:", type(response))
print("Response content:", response.content)
print("Has tool calls:", hasattr(response, 'tool_calls'))
print("Tool calls:", getattr(response, 'tool_calls', None))

In [ ]:
print("Tool name:", retrieve.name)
print("Tool description:", retrieve.description)
print("Tool args:", retrieve.args)

In [ ]:
try:
    llm_forced = llm.bind_tools([retrieve], tool_choice="any")
    response = llm_forced.invoke([HumanMessage(content="Find 1980s sci-fi movies")])
    print("Forced tool choice - Tool calls:", getattr(response, 'tool_calls', None))
except Exception as e:
    print(f"Tool choice failed: {e}")

# Try with a very explicit message
explicit_msg = HumanMessage(content="I need you to call the retrieve function with the query '1980s sci-fi movies'. Please use the available tool.")
response = llm_with_tools.invoke([explicit_msg])
print("Explicit message - Tool calls:", getattr(response, 'tool_calls', None))